In [2]:
import numpy
from math import pi
%matplotlib inline

from laplace_helper import plot_3D, L2_rel_error
from cg_helper import poisson_2d, p_analytical

In [3]:
nx = 101
ny = 101
xmin = 0
xmax = 1
ymin = -0.5
ymax = 0.5

l2_target = 1e-10

dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

x= numpy.linspace(xmin, xmax, nx)
y = numpy.linspace(ymin, ymax, ny)
X,Y = numpy.meshgrid(x,y)

#Source
L = xmax-xmin
b = -2*(pi/L)**2*numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)

#Initialization
p_i = numpy.zeros((ny, nx))

#Analytical solution
pan = p_analytical(X,Y,L)

In [4]:
def steepest_descent_2d(p, b, dx, dy, l2_target):
    '''Performs steepest descent relaxation 
        Assumes Dirichlet boundary conditions p=0
    
    Parameters:
        p- initial guess
        b- source term
        dx- mesh spacing
        dy- mesh spacing
        l2_target- exit criteria
        
    Returns:
        p - distribution after relaxation
        
    '''
    ny, nx = p.shape
    r = numpy.zeros((ny, nx))
    Ar = numpy.zeros((ny, nx))
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    while l2_norm > l2_target:
        pd = p.copy()
        r[1:-1, 1:-1] = b[1:-1, 1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:,1:-1] - pd[:-2, 1:-1]
        
        Ar[1:-1, 1:-1] = -4*r[1:-1, 1:-1] + r[1:-1,2:]+r[1:-1,:-2]+\
            r[2:,1:-1] + r[:-2, 1:-1]
            
        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma
        
        p = pd+ alpha*r
        
        l2_norm = L2_rel_error(pd, p)
        iterations +=1
        l2_conv.append(l2_norm)
        
    print('Number of SD iterations: {0:d}'.format(iterations))
    return p, l2_conv

In [5]:
p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p, pan)

Number of SD iterations: 2


8.2250762212767885e-05

In [6]:
def conjugate_gradient_2d(p, b, dx, dy, l2_target):
    '''Performs Conjugate Gradient relaxation
        Assumes Dirichlet boundary conditions p=0
        
        Parameters:
            p- intial guess
            b- source term
            dx- mesh spacing in x
            dy- mesh spacing in y
            l2_target- exit criteria
            
        Returns:
            p- distribution after relaxation
    '''
    
    ny, nx = p.shape
    r = numpy.zeros((ny, nx))
    Ad = numpy.zeros((ny, nx))
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    #compute intial residual, which we will use for the first direction 
    r[1:-1, 1:-1] = b[1:-1, 1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:,1:-1] - p[:-2,1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2] + \
        d[2:,1:-1] + d[:-2,1:-1]
    sigma = numpy.sum(d*Ad)
    
    #Iterations
    while l2_norm > l2_target:
        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        alpha = rho/sigma
        
        p = pk + alpha*dk
        r = rk - alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1, 1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:,1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
        
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv

In [7]:
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p, pan)

Number of CG iterations: 2


8.2250762212769755e-05

In [8]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31227


In [9]:
b = (numpy.sin(pi*X/L)*numpy.cos(pi*Y/L) + numpy.sin(6*pi*X/L)*numpy.cos(6*pi*Y/L))

In [10]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)
p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31226
Number of SD iterations: 31705
Number of CG iterations: 72
